In [5]:
import numpy
from matplotlib import pyplot, cm
from mpl_toolkits.mplot3d import Axes3D

import time

In [6]:
@njit(fastmath=True)
def solve_2d_Burger():
    ###variable declarations
    nx = 41
    ny = 41
    nt = 420
    dx = 2 / (nx - 1)
    dy = 2 / (ny - 1)
    sigma = .0009
    nu = 0.01
    dt = sigma * dx * dy / nu
    
    
    x = numpy.linspace(0, 2, nx)
    y = numpy.linspace(0, 2, ny)
    
    u = numpy.ones((ny, nx))  # create a 1xn vector of 1's
    v = numpy.ones((ny, nx))
    un = numpy.ones((ny, nx)) 
    vn = numpy.ones((ny, nx))
    
    ###Assign initial conditions
    
    ##set hat function I.C. : u(.5<=x<=1 && .5<=y<=1 ) is 2
    u[int(.5 / dy):int(1 / dy + 1),int(.5 / dx):int(1 / dx + 1)] = 2 
    ##set hat function I.C. : u(.5<=x<=1 && .5<=y<=1 ) is 2
    v[int(.5 / dy):int(1 / dy + 1),int(.5 / dx):int(1 / dx + 1)] = 2
    row, col = u.shape
    
        
    #fill the updating of u and v
    #..    
    
    return x, y, u


In [7]:
#compute the cpu time of the solving 2d Burgers equation
x, y, u = solve_2d_Burger()

In [8]:
#Plotting data